In [1]:
include("demands_adjustment_Sioux_simp_gradi.jl")
include("tap_Sioux_simp.jl")

TAP (generic function with 1 method)

In [2]:
# load Jacobian
jacobian = readall("jacobian.json");
jacobian = JSON.parse(jacobian);

jacob = zeros(numODpairs, numLinks)
for i = 1:numLinks
    for j = 1:numODpairs
        jacob[j, i] = jacobian[i][j]
    end
end

In [3]:
jacob

132x30 Array{Float64,2}:
 -0.0   0.0   0.0  0.0   0.0  -0.0  0.0  …   1.0  -0.0  0.0   0.0  0.0   1.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0  -1.0   0.0  0.0   0.0   0.0  0.0      1.0   0.0  0.0   1.0  0.0   0.0
 -1.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   1.0  0.0      0.0   1.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0  …   1.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0  -0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0  -0.0  0.0   0.0
 -1.0   0.0   1.0  0.0   0.0   0.0  0.0      0.0   1.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
 -1.0   0.0   0.0  0.0   0.0   0.0  0.0  …   0.0   0.0  0.0   0.0  0.0   1.0
  0.0   0.0   1.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0  -0.0   0.0  0.0     -0.0   

In [31]:
x, x_0

([2070.0,3510.0,2070.0,3150.0,3510.0,6240.91,2849.09,6284.98,9810.0,3900.91  …  7110.0,4410.0,12150.0,12150.0,7110.0,3854.98,7200.0,3885.02,2805.02,3929.09],[2300.0,3900.0,2300.0,3500.0,3900.0,7060.55,3039.45,7105.27,10650.3,4202.51  …  7900.0,5149.72,13758.0,13749.7,8158.04,4155.55,8249.72,4194.73,2994.73,4239.45])

In [4]:
# compute the gradient
gradi_ = zeros(numODpairs)
for i = 1:numODpairs
    gradi_[i] = sum([2 * (x[j] - x_0[j]) * jacob[i, j] for j = 1:numLinks])
end

In [5]:
numNodes

12

In [6]:
OD_pair_label_dict = Dict()
OD_pair_label_dict_ = Dict()
label = 1
for i = 1:numNodes
    for j = 1:numNodes
        if i != j
            key = "($(i), $(j))"
            OD_pair_label_dict[key] = label
            OD_pair_label_dict_[label] = key
            label += 1
        end
    end
end

In [7]:
OD_pair_label_dict_[numODpairs]

"(12, 11)"

In [8]:
#load OD pair labels
odPairLabel = readall("od_pair_label_dict_Sioux_simp_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__Sioux_simp_refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

In [9]:
# ensure the consistency of dimensions

gradi = zeros(numODpairs)

for i = 1:numODpairs
    if ~("($(odPairLabel_["$(i)"][1]), $(odPairLabel_["$(i)"][2]))" in keys(OD_pair_label_dict))
        gradi[i] = 0
    else
        gradi[i] = gradi_[OD_pair_label_dict["($(odPairLabel_["$(i)"][1]), $(odPairLabel_["$(i)"][2]))"]]
    end
end

In [10]:
length(demands)

144

In [12]:
# for convenience of computing the new demand vector
demandsVec = zeros(length(odPairLabel_))

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [13]:
demandsVec

132-element Array{Float64,1}:
   90.0
   90.0
  450.0
  180.0
  270.0
  450.0
  720.0
  450.0
 1170.0
  450.0
  180.0
   90.0
   90.0
    ⋮  
 1260.0
  180.0
   90.0
  180.0
  540.0
  180.0
  180.0
  630.0
  540.0
  540.0
 1800.0
 1260.0

In [14]:
demands

Dict{(Int64,Int64),Float64} with 144 entries:
  (8,10)  => 1440.0
  (9,5)   => 720.0
  (3,6)   => 270.0
  (10,5)  => 900.0
  (6,9)   => 360.0
  (12,4)  => 540.0
  (4,4)   => 0.0
  (9,4)   => 630.0
  (10,11) => 3600.0
  (8,9)   => 720.0
  (9,1)   => 450.0
  (11,1)  => 450.0
  (8,11)  => 720.0
  (3,1)   => 90.0
  (1,10)  => 1170.0
  (1,12)  => 180.0
  (12,7)  => 630.0
  (4,5)   => 450.0
  (9,9)   => 0.0
  (11,8)  => 720.0
  (12,8)  => 540.0
  (2,4)   => 180.0
  (6,5)   => 180.0
  (4,9)   => 630.0
  (8,4)   => 630.0
  ⋮       => ⋮

In [15]:
# convert the demand data into dictionary; for convenience of solving the TAP problem and the inverse VI problem

demandsDict = Dict{(Int64,Int64), Float64}()

for key = [(int(split(key, ", ")[1][2:end]), int(split(key, ", ")[2][1:end-1])) for key = keys(odPairLabel)]
    demandsDict[key] = demandsVec[odPairLabel["($(key[1]), $(key[2]))"]]
end

In [16]:
demandsDict

Dict{(Int64,Int64),Float64} with 132 entries:
  (3,8)   => 180.0
  (9,5)   => 720.0
  (3,6)   => 270.0
  (10,5)  => 900.0
  (6,9)   => 360.0
  (12,4)  => 540.0
  (11,1)  => 450.0
  (9,4)   => 630.0
  (10,11) => 3600.0
  (8,9)   => 720.0
  (9,1)   => 450.0
  (8,11)  => 720.0
  (3,1)   => 90.0
  (1,10)  => 1170.0
  (1,12)  => 180.0
  (12,7)  => 630.0
  (4,5)   => 450.0
  (11,8)  => 720.0
  (12,8)  => 540.0
  (2,4)   => 180.0
  (6,5)   => 180.0
  (4,9)   => 630.0
  (8,4)   => 630.0
  (9,2)   => 180.0
  (1,2)   => 90.0
  ⋮       => ⋮

In [17]:
gradi

132-element Array{Float64,1}:
  -4212.86      
  -6880.0       
  -4296.04      
    460.0       
 -11983.8       
  -3396.07      
      0.0       
  -3845.07      
  -3479.45      
     -0.0       
  -3404.65      
  -3400.59      
      0.0       
      ⋮         
  -3621.27      
     -0.0       
  -3359.96      
     -0.0       
  -1919.41      
  -8198.69      
  -2096.07      
      2.46018e-9
  -2700.43      
  -8755.48      
  -4640.72      
  -6483.37      

In [18]:
# compute a descent direction

r = similar(gradi)
for i = 1:length(gradi)
    r[i] = -1 * gradi[i]
end

In [19]:
r

132-element Array{Float64,1}:
  4212.86      
  6880.0       
  4296.04      
  -460.0       
 11983.8       
  3396.07      
    -0.0       
  3845.07      
  3479.45      
     0.0       
  3404.65      
  3400.59      
    -0.0       
     ⋮         
  3621.27      
     0.0       
  3359.96      
     0.0       
  1919.41      
  8198.69      
  2096.07      
    -2.46018e-9
  2700.43      
  8755.48      
  4640.72      
  6483.37      

In [20]:
# compute a search direction

r_ = similar(r)
for i = 1:length(r)
    if (demandsVec[i] > 20) || (demandsVec[i] <= 20 && r[i] > 0)
        r_[i] = r[i]
    else
        r_[i] = 0
    end
end

In [21]:
r_

132-element Array{Float64,1}:
  4212.86      
  6880.0       
  4296.04      
  -460.0       
 11983.8       
  3396.07      
    -0.0       
  3845.07      
  3479.45      
     0.0       
  3404.65      
  3400.59      
    -0.0       
     ⋮         
  3621.27      
     0.0       
  3359.96      
     0.0       
  1919.41      
  8198.69      
  2096.07      
    -2.46018e-9
  2700.43      
  8755.48      
  4640.72      
  6483.37      

In [22]:
# line search

alphaList = Float64[]

for i = 1:length(r_)
    if r_[i] < 0
        push!(alphaList, -demandsVec[i]/r_[i])
    end
end

alpha_max = minimum(alphaList)

0.30527055337549003

In [23]:
length(r_)

132

In [24]:
# update

demandsVecNew = similar(demandsVec)
for i = 1:length(demandsVec)
    demandsVecNew[i] = demandsVec[i] + (alpha_max / 81) * r_[i] 
end

In [25]:
demandsVecNew

132-element Array{Float64,1}:
  105.877
  115.929
  466.191
  178.266
  315.164
  462.799
  720.0  
  464.491
 1183.11 
  450.0  
  192.831
  102.816
   90.0  
    ⋮    
 1273.65 
  180.0  
  102.663
  180.0  
  547.234
  210.899
  187.9  
  630.0  
  550.177
  572.997
 1817.49 
 1284.43 

In [26]:
# convert the demand data into dictionary; for convenience of solving the TAP problem and the inverse VI problem

demandsDictNew = Dict{(Int64,Int64), Float64}()

for key = [(int(split(key, ", ")[1][2:end]), int(split(key, ", ")[2][1:end-1])) for key = keys(odPairLabel)]
    demandsDictNew[key] = demandsVecNew[odPairLabel["($(key[1]), $(key[2]))"]]
end

for i=1:numNodes
    key = (i,i)
    demandsDictNew[key] = 0
end

In [27]:
demandsDictNew

Dict{(Int64,Int64),Float64} with 144 entries:
  (3,8)   => 189.18099995034132
  (9,5)   => 727.7635325707039
  (3,6)   => 286.59801996262894
  (10,5)  => 908.5175636107699
  (6,9)   => 364.82389020427894
  (12,4)  => 547.2338152540801
  (4,4)   => 0.0
  (11,1)  => 447.4372348608614
  (9,4)   => 631.7336352419937
  (10,11) => 3615.1625620457685
  (8,9)   => 720.0
  (9,1)   => 450.0
  (8,11)  => 754.3570163028303
  (3,1)   => 95.36423349586329
  (1,10)  => 1183.1132382802066
  (1,12)  => 192.83133689609093
  (12,7)  => 629.9999999999907
  (4,5)   => 458.7483476780465
  (11,8)  => 722.638140584533
  (9,9)   => 0.0
  (12,8)  => 550.1772949767255
  (2,4)   => 191.9093203529304
  (6,5)   => 206.3649916121843
  (4,9)   => 653.8219651643093
  (8,4)   => 635.954660177466
  ⋮       => ⋮

In [28]:
tapFlowVec = TAP(demands)[1]

([2070.0,3510.0,2070.0,3150.0,3510.0,6240.91,2849.09,6284.98,9810.0,3900.91  …  7110.0,4410.0,12150.0,12150.0,7110.0,3854.98,7200.0,3885.02,2805.02,3929.09],12,30,132)